In [2]:
import numpy as np
import polars as pl

In [3]:
%%time
df = (
	pl.read_csv("/Users/sonle/Documents/Data/delivery_orders_march.csv")
		.select(pl.all().shrink_dtype())
)


CPU times: user 4.86 s, sys: 4.77 s, total: 9.63 s
Wall time: 3.95 s


In [4]:
WORKDAYS = '1111110'
HOLIDAYS = ['2020-03-08','2020-03-25', '2020-03-30', '2020-03-31']

GMT8_OFFSET = 3600 * 8
DURATION_1DAY = 3600 * 24

sla_matrix_1st_attempt = [
	[3, 5, 7, 7],
	[5, 5, 7, 7],
	[7, 7, 7, 7],
	[7, 7, 7, 7]
]

sla_matrix_2nd_attempt =[
	[3, 3, 3, 3],
	[3, 3, 3, 3],
	[3, 3, 3, 3],
	[3, 3, 3, 3]
]
locations = ["Metro Manila", "Luzon", "Visayas", "Mindanao"]
locations = [loc.lower() for loc in locations]
location_to_index = {loc: i for i, loc in enumerate(locations)}
print(location_to_index)

min_length = min(map(len, locations))
trunc_location_to_index = {loc[-min_length:]: i for i, loc in enumerate(locations)}
print(trunc_location_to_index)
#transform to native python type for easily mapping
map_to_dict = dict(enumerate(np.array(sla_matrix_1st_attempt).flatten().tolist()))
print(map_to_dict)

{'metro manila': 0, 'luzon': 1, 'visayas': 2, 'mindanao': 3}
{'anila': 0, 'luzon': 1, 'sayas': 2, 'danao': 3}
{0: 3, 1: 5, 2: 7, 3: 7, 4: 5, 5: 5, 6: 7, 7: 7, 8: 7, 9: 7, 10: 7, 11: 7, 12: 7, 13: 7, 14: 7, 15: 7}


In [6]:
# This code uses predictate condition in polars to map data with given index
%%time
(
	df.with_columns([
		pl.when(pl.col('buyeraddress').str.slice(-min_length).str.to_lowercase() == "anila")
		.then(pl.lit(0))
		.when(pl.col('buyeraddress').str.slice(-min_length).str.to_lowercase() == "luzon")
		.then(pl.lit(1))
		.when(pl.col('buyeraddress').str.slice(-min_length).str.to_lowercase() == "sayas")
		.then(pl.lit(2))
		.otherwise(pl.lit(3))
		.alias('buyer_index')
	])
	.drop('buyeraddress')
)

In [7]:
def replace(column: str, mapping: dict) -> pl.internals.expr.Expr:
	"""
	Function to map cities to indexes
	:param column: columns of polars.Dataframe
	:param mapping: a dictionary of cities and indexes
	:return: mapped Dataframe
	"""
	if not mapping:
		raise Exception("mapping can't be empty")
	elif not isinstance(mapping, dict):
		TypeError(f"maping must be of type dict, but is type: {type(mapping)}")
	if not isinstance(column, str):
		raise TypeError(f"column must be of type str, but is type: {type(column)}")

	#initiate the expression with pl.when
	branch = pl.when(pl.col(column) == list(mapping.keys())[0]) \
				.then(list(mapping.values())[0])
	#for every value add a when.then
	for from_val, to_val in mapping.items():
		branch = branch.when(pl.col(column) == from_val) \
						.then(to_val)

	return branch.otherwise(list(mapping.values()) [-1]).alias(column)


shape: (3176313, 6)
┌────────────────┬────────────┬─────────────────┬────────────────┬────────────────┬────────────────┐
│ orderid        ┆ pick       ┆ 1st_deliver_att ┆ 2nd_deliver_at ┆ buyeraddress   ┆ selleraddress  │
│ ---            ┆ ---        ┆ empt            ┆ tempt          ┆ ---            ┆ ---            │
│ i64            ┆ i32        ┆ ---             ┆ ---            ┆ str            ┆ str            │
│                ┆            ┆ f32             ┆ f32            ┆                ┆                │
╞════════════════╪════════════╪═════════════════╪════════════════╪════════════════╪════════════════╡
│ 2215676524     ┆ 1583138397 ┆ 1.5834e9        ┆ null           ┆ Baging ldl BUE ┆ Pantranco      │
│                ┆            ┆                 ┆                ┆ NAVISTA,PATAG. ┆ vill. 417      │
│                ┆            ┆                 ┆                ┆ CAGA...        ┆ Warehouse#     │
│                ┆            ┆                 ┆                ┆     

In [8]:
print(df)

In [9]:
def convert_time_date(column):
	return (
		pl.col(column)
		.map(lambda x: (x + GMT8_OFFSET)/DURATION_1DAY)
		.cast(pl.Int32)
		.alias(column)
	)

In [10]:
def compute_working_days(df):
	t1 = df.select(convert_time_date("pick")).to_numpy().astype('datetime64[D]')
	t2 = df.select(convert_time_date("1st_deliver_attempt")).to_numpy().astype('datetime64[D]')
	t3 = df.select(convert_time_date("2nd_deliver_attempt").fill_null(strategy='zero')).to_numpy().astype('datetime64[D]')

	num_days1 = np.busday_count(t1, t2, weekmask=WORKDAYS, holidays=HOLIDAYS).flatten()
	num_days2 = np.busday_count(t2, t3, weekmask=WORKDAYS, holidays=HOLIDAYS).flatten()
	return num_days1, num_days2



In [11]:
num_days1, num_days2 = compute_working_days(df)

CPU times: user 2.49 s, sys: 89.8 ms, total: 2.58 s
Wall time: 1.56 s


In [12]:
%%time
result = (df
		  .with_columns([
						pl.col("buyeraddress").str.slice(-min_length).str.to_lowercase(),
						pl.col("selleraddress").str.slice(-min_length).str.to_lowercase(),
						pl.col("2nd_deliver_attempt").fill_null(0).cast(pl.Int32)])
		  .with_columns([replace("buyeraddress", trunc_location_to_index),
						 replace("selleraddress", trunc_location_to_index)])
		  .with_columns((4 * pl.col("buyeraddress") + pl.col("selleraddress")).cast(pl.Int32).alias("sla"))
		  .with_columns([replace("sla", map_to_dict),
						 convert_time_date("pick"),
						 convert_time_date("1st_deliver_attempt"),
						 convert_time_date("2nd_deliver_attempt"),
						 pl.Series(name='num_days1', values=num_days1),
						 pl.Series(name='num_days2', values=num_days2)])
		  )



orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,sla,num_days1,num_days2
i64,i32,i32,i32,i32,i32,i32,i64,i64
2215676524,18323,18326,0,0,0,3,3,-15708
2219624609,18325,18327,18331,0,0,3,2,3
2220979489,18325,18327,0,0,0,3,2,-15709
2221066352,18326,18328,0,3,0,7,2,-15710
2222478803,18325,18327,0,1,0,5,2,-15709
2222597288,18325,18328,0,0,0,3,3,-15710
2222738456,18323,18326,18330,0,3,7,3,3
2224695304,18323,18331,0,0,3,7,7,-15712
2224704587,18325,18326,18330,1,0,5,1,3


In [ ]:
result

In [ ]:

(
	result
	.with_columns(pl.when((pl.col("num_days1") > pl.col("sla")) | (pl.col("num_days2") > 3))
				  	.then(pl.lit(1, pl.Int32))
				  	.otherwise(pl.lit(0, pl.Int32))
				  	.alias("is_late"))
	.select(["orderid", "is_late"])
	.write_csv('submission_polars')
)